# Constructing TensorFlow Recommenders Model

In [2]:
import tensorflow_recommenders as tfrs

2023-06-14 09:47:04.292178: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import pandas as pd
import numpy as np

In [4]:
import string

In [5]:
fiverr_df = pd.read_csv('/Users/rsoedarnadi/Documents/GitHub/machine-learning/fiverr_data.csv', delimiter=',')

In [5]:
fiverr_df.head(6)

,illustrator_name,gig_rating,gig_price_from,gig_img_url,gig_url,gig_design_type,gig_design_style,client_origins,client_names,illustrator_origins
0,nadia_younus,5.0,60,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/nadia_younus/do-minimal...,logos,minimalist,Canada,stoxventures,Pakistan
1,mariya_logo99,4.9,15,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/mariya_logo99/do-creati...,logos,minimalist,Italy,lifehossain,Bangladesh
2,mountdesign,4.9,10,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/mountdesign/design-3-pr...,logos,minimalist,United States,stormproofrooof,India
3,ahmed_gfx786,4.9,5,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/ahmed_gfx786/design-a-u...,logos,minimalist,France,remi_bgd,Pakistan
4,websutra,4.9,400,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/websutra/draw-an-unique...,logos,minimalist,United States,tribolumen,Sri Lanka
5,kiara43,4.9,5,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/kiara43/create-modern-m...,logos,minimalist,Germany,sustainifytoday,Sri Lanka


In [6]:
fiverr_df = fiverr_df.dropna().drop_duplicates().reset_index(drop=True)

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
price = fiverr_df[['gig_price_from']]

In [9]:
scaler = MinMaxScaler()

In [10]:
price=scaler.fit_transform(price)

In [11]:
fiverr_df['gig_price_from']=price

In [12]:
fiverr_df['gig_rating']=np.asarray(fiverr_df['gig_rating'],dtype=np.float64) / 5

In [115]:
cos_sim_df = fiverr_df[['gig_img_url','gig_design_type','gig_design_style', 'illustrator_origins']]

In [116]:
def remove_punc(text):
    cleaned = text.translate(str.maketrans('','', string.punctuation)).lower()
    return cleaned

In [117]:
cos_sim_df['illustrator_origins']=cos_sim_df['illustrator_origins'].apply(remove_punc)

/var/folders/ht/1vg09v7x16x7sbqnmzfvc4600000gn/T/ipykernel_73751/2612384505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cos_sim_df['illustrator_origins']=cos_sim_df['illustrator_origins'].apply(remove_punc)


In [118]:
cos_sim_df['joined_feat'] = ''
cos_sim_df['joined_feat'] = cos_sim_df[cos_sim_df.columns[1:]].apply(lambda x: ' '.join(x), axis=1)

/var/folders/ht/1vg09v7x16x7sbqnmzfvc4600000gn/T/ipykernel_73751/2163304710.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cos_sim_df['joined_feat'] = ''
/var/folders/ht/1vg09v7x16x7sbqnmzfvc4600000gn/T/ipykernel_73751/2163304710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cos_sim_df['joined_feat'] = cos_sim_df[cos_sim_df.columns[1:]].apply(lambda x: ' '.join(x), axis=1)


In [119]:
cos_sim_df.set_index('gig_img_url', inplace=True)

In [120]:
cos_sim_df=cos_sim_df[[ 'joined_feat']]

In [121]:
cos_sim_df.head()

,joined_feat
gig_img_url,
"https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/192437271/original/3fb8af5466dc09f13e2afaccf22c6708f0a8976e.jpg",logos minimalist pakistan
"https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/292635463/original/e6d48abe79bee1a9cdc2c71e83c88513440fc393.jpg",logos minimalist bangladesh
"https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/109611151/original/42d4814c676b9f4320152d541bb6c8be7cd83d39.jpg",logos minimalist india
"https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/132488260/original/955d6d66789bd57075f25da7de2b3a4233b2e26a.jpg",logos minimalist pakistan
"https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/122531603/original/eccdc57ef960b6cc51853d37a1ff9034af618deb.jpg",logos minimalist sri lanka


In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [124]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(cos_sim_df['joined_feat'])

In [125]:
from sklearn.metrics.pairwise import cosine_similarity

In [126]:
cos_sim = cosine_similarity(tfidf_matrix)

In [127]:
cos_sim.shape

(1686, 1686)

In [202]:
def predict(gig_img_url_, top_n=20):
    data = cos_sim_df.reset_index()
    final_df = pd.concat([fiverr_df[['gig_rating', 'gig_price_from']], data], axis=1)
    gig_index = data[data['gig_img_url'] == gig_img_url_].index
    similarity = cos_sim[gig_index[0]].T
    
    sim_df = pd.DataFrame(similarity, columns=['similarity'])
    final_df = pd.concat([final_df, sim_df], axis=1)
    # You can also play around with the number
    final_df['gig_rating'] = np.round(final_df['gig_rating'][gig_index[0]] * final_df['similarity'],1)
    final_df=final_df.drop(gig_index[0])
    final_df_sorted = final_df.sample(100).sort_values(by='gig_rating', ascending=False).head(top_n)
    final_df_sorted['client_names'] = fiverr_df['client_names'][gig_index[0]]
    final_df_sorted = final_df_sorted[['gig_img_url', 'client_names', 'gig_rating', 'gig_price_from']]
    
    return final_df_sorted

In [203]:
for_test_df=predict(fiverr_df['gig_img_url'][400])

In [204]:
for_test_df

,gig_img_url,client_names,gig_rating,gig_price_from
413,https://fiverr-res.cloudinary.com/t_gig_cards_...,dgreendigital,3.1,20
381,https://fiverr-res.cloudinary.com/t_gig_cards_...,dgreendigital,3.1,55
385,https://fiverr-res.cloudinary.com/t_gig_cards_...,dgreendigital,2.9,75
536,https://fiverr-res.cloudinary.com/t_gig_cards_...,dgreendigital,2.9,30
10,https://fiverr-res.cloudinary.com/t_gig_cards_...,dgreendigital,2.9,45
98,https://fiverr-res.cloudinary.com/t_gig_cards_...,dgreendigital,2.9,15
388,https://fiverr-res.cloudinary.com/t_gig_cards_...,dgreendigital,2.7,10
343,https://fiverr-res.cloudinary.com/t_gig_cards_...,dgreendigital,2.3,35
710,https://fiverr-res.cloudinary.com/t_gig_cards_...,dgreendigital,2.2,5
1048,https://fiverr-res.cloudinary.com/t_gig_cards_...,dgreendigital,2.1,5


In [264]:
for_model_df=fiverr_df[['gig_img_url', 'client_names', 'gig_rating', 'gig_price_from']]

In [265]:
for i in range(len(fiverr_df)):
    pred_df=predict(fiverr_df['gig_img_url'][i])
    for_model_df=pd.concat([for_model_df, pred_df], axis=0)
    print('success '+str(i))

success 0
success 1
success 2
success 3
success 4
success 5
success 6
success 7
success 8
success 9
success 10
success 11
success 12
success 13
success 14
success 15
success 16
success 17
success 18
success 19
success 20
success 21
success 22
success 23
success 24
success 25
success 26
success 27
success 28
success 29
success 30
success 31
success 32
success 33
success 34
success 35
success 36
success 37
success 38
success 39
success 40
success 41
success 42
success 43
success 44
success 45
success 46
success 47
success 48
success 49
success 50
success 51
success 52
success 53
success 54
success 55
success 56
success 57
success 58
success 59
success 60
success 61
success 62
success 63
success 64
success 65
success 66
success 67
success 68
success 69
success 70
success 71
success 72
success 73
success 74
success 75
success 76
success 77
success 78
success 79
success 80
success 81
success 82
success 83
success 84
success 85
success 86
success 87
success 88
success 89
success 90
success 9

In [266]:
for_model_df.shape

(35406, 4)

In [220]:
for_model_df.to_csv('/Users/rsoedarnadi/Downloads/for_model_df.csv')

In [208]:
from typing import Dict, Text

import tensorflow as tf
import tensorflow_recommenders as tfrs

In [289]:
ratings = tf.data.Dataset.from_tensor_slices(dict(fiverr_df[['gig_img_url','client_names', 'gig_rating']]))
imgs = tf.data.Dataset.from_tensor_slices(dict(fiverr_df[['gig_img_url']]))

In [270]:
import pprint
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'client_names': b'stoxventures',
 'gig_img_url': b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_a'
                b'uto/gigs/192437271/original/3fb8af5466dc09f13e2afaccf22c6708'
                b'f0a8976e.jpg',
 'gig_price_from': 60,
 'gig_rating': 5.0}


2023-06-14 13:11:13.149457: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [35406]
	 [[{{node Placeholder/_3}}]]


In [290]:
ratings = ratings.map(lambda x: {
    "gig_img_url": x["gig_img_url"],
    "client_names": x["client_names"],
    "gig_rating": x["gig_rating"]
})

imgs = imgs.map(lambda x: x["gig_img_url"])

In [291]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(35_406, seed=42, reshuffle_each_iteration=False)
train = ratings.take(30_000)
test = ratings.skip(30_000).take(5_406)

In [292]:
img_urls = imgs.batch(1_000)
clients_names = ratings.batch(1_000_000).map(lambda x: x['client_names'])

unique_img_urls = np.unique(np.concatenate(list(img_urls)))
unique_clients_names = np.unique(np.concatenate(list(clients_names)))

unique_img_urls[:10]

2023-06-14 13:48:38.438936: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1686]
	 [[{{node Placeholder/_0}}]]
2023-06-14 13:48:38.463150: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [1686]
	 [[{{node Placeholder/_2}}]]


array([b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/100175685/original/0531df1a9873ba7d4bfa28831916295c8996b355.png',
       b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/100267701/original/dc68a5ca86285ba97ed2fdbac7eb8118fb79b129.jpg',
       b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/100633290/original/d2e5623413c4650fd54972f6accc5ed33c5d8144.jpg',
       b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/100752175/original/1e8500b760079bbeb8d57e8d09c979fb6cb6a478.jpg',
       b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/100849695/original/861139cd50b2e53d18e301ae12619c2d397ee171.jpg',
       b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/100955264/original/0500c21e89e1349e2cea19cdcb7f64c9dede9ba3.jpg',
       b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/101019442/original/e532854981e67059236c848215363fc0d5d175f

In [293]:
embedding_dimension = 32

client_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_clients_names, mask_token=None),
        tf.keras.layers.Embedding(len(unique_clients_names) + 1,   
                                  embedding_dimension)
])

item_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_img_urls, mask_token=None),
        tf.keras.layers.Embedding(len(unique_img_urls) + 1, 
                                  embedding_dimension)
])

In [294]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=imgs.batch(128).map(item_model)
)

task = tfrs.tasks.Retrieval(
  metrics=metrics
)


In [295]:
class Model(tfrs.Model):
  def __init__(self, client_model, item_model):
    super().__init__()
    self.item_model: tf.keras.Model = item_model
    self.client_model: tf.keras.Model = client_model
    self.retrieval_task: tf.keras.layers.Layer = task
    
  
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    client_embeddings = self.client_model(features['client_names'])
    positive_item_embeddings = self.item_model(features['gig_img_url'])

    # The task computes the loss and the metrics.

    return self.retrieval_task(client_embeddings, positive_item_embeddings)

In [301]:
tf.keras.backend.clear_session()

In [302]:
model = Model(client_model, item_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [303]:
cached_train = train.shuffle(1_686).batch(300).cache()
cached_test = test.batch(300).cache()

In [304]:
model.fit(cached_train, epochs=10)

Epoch 1/10


2023-06-14 13:51:29.705092: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1686]
	 [[{{node Placeholder/_0}}]]


2/6 [=========>....................] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.9967 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1690.9518 - regularization_loss: 0.0000e+00 - total_loss: 1690.9518

2023-06-14 13:51:30.680403: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [300,32]
	 [[{{node Placeholder/_12}}]]
2023-06-14 13:51:30.771639: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1686]
	 [[{{node Placeholder/_0}}]]
2023-06-14 13:51:30.851782: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [3

6/6 [==============================] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.9911 - factorized_top_k/top_10_categorical_accuracy: 0.9982 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1568.6976 - regularization_loss: 0.0000e+00 - total_loss: 1568.6976

...

2023-06-14 13:51:34.973328: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [186,32]
	 [[{{node Placeholder/_12}}]]
2023-06-14 13:51:35.063143: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype resource
	 [[{{node Placeholder/_6}}]]
2023-06-14 13:51:35.136968: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype resource
	 [[{{node Placeholder/_6

4/6 [===================>..........] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 854.8870 - regularization_loss: 0.0000e+00 - total_loss: 854.8870

2023-06-14 13:51:35.218445: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [300,32]
	 [[{{node Placeholder/_12}}]]
2023-06-14 13:51:35.296525: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1686]
	 [[{{node Placeholder/_0}}]]
2023-06-14 13:51:35.373204: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype resource
	 [[{{node

6/6 [==============================] - 1s 96ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 739.3718 - regularization_loss: 0.0000e+00 - total_loss: 739.3718


2023-06-14 13:51:35.503743: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [186,32]
	 [[{{node Placeholder/_12}}]]


In [306]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.client_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((imgs.batch(100), imgs.batch(100).map(model.item_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["ismetrix"]))
print(f"Recommendations for user: {titles[0, :10]}")

Recommendations for user: [b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/106187992/original/12afe768e2ba9af5f211d0358dff19f6e419d38d.jpg'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/55296829/original/a2df98e738e8d48b3be14f33a1d9f31c0fd4a374.jpg'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/200141529/original/c236815cf390362bf52b3507132d740f59202a5c.jpg'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/245231687/original/1681fc67722b5f2562c143377b8de68ab900c582.jpg'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/289425037/original/a60dbecfe51fb5dc9836328b535f636f90ec3875.png'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/169342074/original/9f79b1169bb6375f12281af90e0219a19920e5a9.jpg'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/125309859/original/89c36a4941eb1608e14b6363f2469c90c2449b02.jpg'
 b'https://fiver

2023-06-14 13:52:42.720226: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1686]
	 [[{{node Placeholder/_0}}]]
2023-06-14 13:52:42.720720: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_13' with dtype int64
	 [[{{node Placeholder/_13}}]]


In [61]:
model.save_weights('tfrs.h5')